In [ ]:
%%time
#############需先将阵列板规范化###################
#将384点板剪裁成384格,按板名命名文件夹保存各自的单藻落

import os 
import cv2
import numpy as np
import pylab
import matplotlib.pyplot as plt


def file_name2(file_dir): #识别路径目录下包含的所有png格式照片
    #注意返回至文件夹中所有文件，会打开深层文件夹查找
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.png':
                L.append(os.path.join(root, file))
    return L

def cv_imread(filePath):#解决中文不识别问题
    cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    return cv_img 




def divide_spot(file_name):
    img = cv_imread(file_name)
    file_name=file_name.split(".png")[0]
    os.mkdir(file_name) 
    #print("图像的形状,返回一个图像的(行数,列数,通道数):",img.shape)
    w=img.shape[1]
    h=img.shape[0]
    h0=h/16
    w0=w/24   
    j=0
    while j<16:
        i=0
        while i<24:
            img_names=file_name+"/"+str(24*j+i+1)+".png"
            a=round(j*h0)#注意，此时仍为四舍五入取法获取区间，仍有一个像素的遗漏或重叠
            b=round((j+1)*h0)
            c=round(i*w0)
            d=round((i+1)*w0)
            img0=img[a:b,c:d]
            #cv2.imwrite(img_names,img0)#不适用python3，cv2.imencode能完整保存
            cv2.imencode('.png', img0)[1].tofile(img_names)
            i+=1
        j+=1
    


for i in file_name2('/path_to_the_picture'):#注意会遍历文件夹下所有文件，小心存储
    if "D"in i:################注意更换
        print(i)
    
    #img = cv_imread(i)
        divide_spot(i)

In [ ]:
%%time
##############存储符合RGB范围的区域值，保存HSV格式###########
import os 
import cv2
import numpy as np
#np.float32
import pylab
import matplotlib.pyplot as plt

def file_name1(file_dir): #识别路径目录下包含的所有‘Run’字符串的文件夹，注意更改
    #注意返回至文件夹中所有文件，会打开深层文件夹查找
    F=[] 
    for root, dirs, files in os.walk(file_dir):
        for foldername in dirs:
            if "D" in foldername:##########################################注意更改########################
                F.append(os.path.join(root, foldername))
    return F

def file_name2(file_dir): #识别路径目录下包含的所有png格式照片
    #注意返回至文件夹中所有文件，会打开深层文件夹查找
    L=[] 
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.png':
                L.append(os.path.join(root, file))
    return L

def cv_imread(filePath):#解决中文不识别问题
    cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    return cv_img 
 

def convert_hsv(file_name,f1,f2): #注意此为位图时，HSV格式
    imgy = cv_imread(file_name)
    img=cv2.cvtColor(imgy,cv2.COLOR_BGR2HSV)#HSV空间
    #gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #img=gray.reshape((-1))

    m=img.shape[0]
    n=img.shape[1]
    m1=0
    while m1<m:
        n1=0
        while n1<n:
            #print(1)
            if len(imgy[m1,n1])==4:
                (b,g,r,h) = imgy[m1,n1]#读取(0,0)像素，Python中图像像素是按H,S,V顺序存储的
            else:(b,g,r) = imgy[m1,n1]
            #print(b1,g1,r1)
            (b1,g1,r1)=img[m1,n1]
            
            #print(b1,g1,r1)
            if b<155 or r>170:#####范围缩小b<155,识别藻落区间的简单方式。
                f2.write(file_name.split("green/")[-1]+","+str(m1)+","+str(n1)+","+str(b)+","+str(g)+","+str(r)+"\n")
                f1.write(file_name.split("green/")[-1]+","+str(m1)+","+str(n1)+","+str(b1)+","+str(g1)+","+str(r1)+"\n")#注意与单块板存储的区别
            n1+=1
        m1+=1

def change_black(image):#将图片中灰色部分转换为黑色，查看识别位置用
    
    img = cv_imread(image)
    m=img.shape[0]
    n=img.shape[1]
    m1=0
    while m1<m:
        n1=0
        while n1<n:
            (b,g,r,h) = img[m1,n1]#读取(0,0)像素，HSV
            if b<190 or g<190 or r<190:#
                #f.write(str(b)+","+str(g)+","+str(r)+","+str(int(b)+int(g)+int(r))+"\n")
                b=0
                g=0
                r=0
                img[m1,n1]=(b,g,r,h)
            n1+=1
        m1+=1
    return img          
        
        

f1=open('g_fhsv.csv','w') 
f1.write("name"+",m1,n1,"+"H"+","+"S"+","+"V"+"\n")
f2=open('g_frgb.csv','w') 
f2.write("name"+",m1,n1,"+"B"+","+"G"+","+"R"+"\n")
for j in file_name1('/storage3/pamgreen20230925/green'):
    print (j)
    if "Ffee/"not in j:
        #print (j)
        for i in file_name2(j):
            convert_hsv(i,f1,f2)
        
        #查看用################
            #template = change_black(i)
            #new_names=i.split(".png")[0]+"_190.png"
            #cv2.imencode('.png', template)[1].tofile(new_names)
        
f1.close()
f2.close()

In [ ]:
%%time
import numpy as np
from scipy import stats
##########获取藻落点个数与众数###############
f1=open('g_fhsv.csv','r')
f2=open('g_fhsvuniq.csv','w')


#np.median(nums)  
#np.mean(nums)  
#C:\Users\JK_GKN\Desktop\imagetest83\g1\1.png,19,23,153,219,202,255
dictseq={}
a=f1.readline().strip()
while 1:
    a=f1.readline().strip()
    if a=="":break
    if "name" not in a:
        b=a.split(",")
        name=str(b[0])
        b1=int(b[3])
        g=int(b[4])
        r=int(b[5])
        if name in dictseq:
            dictseq[name][0].append(b1)
            dictseq[name][1].append(g)
            dictseq[name][2].append(r)
        else:
            dictseq[name]=[[b1],[g],[r]]
    
print(len(dictseq))
f2.write("name,num1,num2,num3,modeh,modes,modev,meanh,means,meanv,medianh,medians,medianv\n")
for key in dictseq:
    f2.write(str(key)+","+str(len(dictseq[key][0]))+","+str(len(dictseq[key][1]))+","+str(len(dictseq[key][2]))+
             ","+str(stats.mode(dictseq[key][0])[0][0])+","+str(stats.mode(dictseq[key][1])[0][0])+","+str(stats.mode(dictseq[key][2])[0][0])+
             ","+str(np.mean(dictseq[key][0]))+","+str(np.mean(dictseq[key][1]))+","+str(np.mean(dictseq[key][2]))+
             ","+str(np.median(dictseq[key][0]))+","+str(np.median(dictseq[key][1]))+","+str(np.median(dictseq[key][2]))+"\n")

f1.close()
f2.close()

print("The end!")

In [ ]:
%%time
import numpy as np
from scipy import stats
##########计算为单一列###############
f2=open('g_fhsvuniquniq.csv','w')
f1=open('g_fhsvuniq.csv','r')
a=f1.readline().strip()
f2.write(a+",value\n")
#f2.write("name,value\n")
#
while 1:
    a=f1.readline().strip()
    if a=="":break
    b=a.split(",")
    c=float(b[1])
    d=float(b[7])
    e=float(b[10])
    f=float(b[12])
    v=0.002*c-20.604*d+12.382*e-6.119*f##############此公式为实验室全员对测试藻落黄绿程度做打分值计算回归
    vv=round(v,3)
    #########添加上原始数据，单一列做筛选相关性较弱
    #f2.write(b[0].split("C:\\Users\\JK_GKN\\Desktop\\green918\\384r\\")[-1]+","+a+","+str(vv)+"\n")
    f2.write(a+","+str(vv)+"\n")
    
    



f1.close()
f2.close()

print("The end!")

In [ ]:
%%time
import numpy as np
from scipy import stats
########################
f2=open('g_fhsvuniqname.csv','w')
f1=open('g_fhsvuniquniq.csv','r')
#################转换数字序列为板的行列序列###########
def changeseq_384(num):
    a=(num-1)//24
    tail=(num-1)%24+1
    head=chr(65+a)
    body=head+str(tail)
    return body
def changeseq_96(num):
    a=(num-1)//12
    tail=(num-1)%12+1
    head=chr(65+a)
    body=head+str(tail)
    return body
#name,num,modev,meanv,medianv,allv
#f2.write("name,num,modev,meanv,medianv,allv\n")
while 1:
    a=f1.readline().strip()
    if a=="":break
    if "name" in a:
        f2.write(a+"\n")
    else:
        #/storage3/green20221124/F1-D10/32.png,303,
        b=int(a.split(".png")[0].split("/")[-1])
        c=a.split("/")[-2]+"-"+changeseq_384(b)
        d=a.split(",")
        f2.write(c+","+",".join(d[1:])+"\n")

f1.close()
f2.close()

print("The end!")

In [ ]:
%%time
############################合并不同组数据#############
######################################################
##################以A1-P24为顺序排列各组数据##########
import numpy as np
from scipy import stats

#f1=open('g_fhsvuniqname.csv','r')
#f3=open('g96_fhsvuniqname.csv','r')
f3=open('g_fhsvuniqname.csv','r')
f2=open('g.csv','w')

def changeseq_384(num):
    a=(num-1)//24
    tail=(num-1)%24+1
    head=chr(65+a)
    body=head+str(tail)
    return body
def dictseq_0(dictseq):
    i=1
    while i<385:
        dictseq[changeseq_384(i)]=changeseq_384(i)+",NA"*13
        i+=1
    return dictseq

    

dictseq={}
b=f3.readline()
f2.write(b)
#F1-D10-A21
while 1:
    a=f3.readline().strip()
    if a=="":break
    b=a.split(",")   
    
    name=b[0].split("-")[0]+"-"+b[0].split("-")[1]
    site=b[0].split("-")[-1]
    if name in dictseq: 
        dictseq[name][site]=a
    else:
        dictseq1={}
        dictseq[name]=dictseq_0(dictseq1)
        dictseq[name][site]=a
    
for key in sorted(dictseq):
    for keys in dictseq[key]:
        if "NA" in dictseq[key][keys]:
            f2.write(key+","+key+"-"+dictseq[key][keys]+"\n")
        else:
            f2.write(key+","+dictseq[key][keys]+"\n")


        

#f1.close()
f2.close()
f3.close()
print("The end!")